In [32]:
# import tensorflow as tf
import pandas as pd
import numpy as np
import gzip
import simplejson

In [ ]:

keys = {i[i.find('/') + 1:]: i for i in ["product/productId",
                                         "review/userId",
                                         "review/profileName",
                                         "review/helpfulness",
                                         "review/score",
                                         "review/time",
                                         "review/summary"]}


def parse(filename):
    f = gzip.open(filename, 'r')
    entry = {}
    for l in f:
        l = str(l.strip())
        colonPos = l.find(':')
        if colonPos == -1:
            yield entry
            entry = {}
            continue
        eName = l[:colonPos][2:]
        rest = l[colonPos+2:-1]
        entry[eName] = rest
    yield entry
    

for i in parse("data/movies.txt.gz"):
    print(i)

{'product/productId': 'B004FM2ENU', 'review/userId': 'AUE2FETR0B2SW', 'review/profileName': 'Joseph Baranowski', 'review/helpfulness': '0/0', 'review/score': '5.0', 'review/time': '1331856000', 'review/summary': 'Great Series!!!', 'review/text': "What a Great Series!!! We watched Season 2 and was so imnpressed that had to buy Season 1! Can't wait to see Season 3! Sceanry, actors and story line all work so well together."}